In [ ]:
import pandas as pd
import openpyxl
from fuzzywuzzy import fuzz, process
import re

# Función del Proyecto Separación
def proyecto_separacion(ruta_entrada, ruta_salida):
    """
    Procesa el archivo Excel para separar descripciones y guarda el resultado en un nuevo archivo.

    Args:
    ruta_entrada (str): Ruta del archivo de entrada.
    ruta_salida (str): Ruta del archivo de salida.
    """
    df = pd.read_excel(ruta_entrada, sheet_name='Hoja1', header=0).applymap(lambda x: str(x).upper())

    def unificar_reemplazar(row):
        descripcion_unificada = ' '.join([str(row[f'Descripción de la Mercancía Detallada {i}']) for i in range(1, 5)])
        descripcion_unificada = descripcion_unificada.replace(row['Separador'], f'|{row["Separador"]}')
        return descripcion_unificada

    df['Descripción Unificada'] = df.apply(unificar_reemplazar, axis=1)
    df_unificado = df[['Número de Aceptación', 'Descripción Unificada']]

    nuevas_descripciones = []
    for index, row in df.iterrows():
        descripciones_divididas = row['Descripción Unificada'].split('|')
        for nueva_descripcion in descripciones_divididas:
            separada = len(descripciones_divididas) > 1
            nuevas_descripciones.append({
                'Descripción': nueva_descripcion.strip(),
                'Número de Aceptación': row['Número de Aceptación'],
                'Separada': separada
            })

    df_nuevo = pd.DataFrame(nuevas_descripciones)
    df_nuevo.to_excel(ruta_salida, index=False)

# Función del Proyecto Depuración
def proyecto_depuracion(ruta_entrada, ruta_salida):
    """
    Normaliza nombres en el archivo Excel y guarda el resultado en un nuevo archivo.

    Args:
    ruta_entrada (str): Ruta del archivo de entrada.
    ruta_salida (str): Ruta del archivo de salida.
    """
    df = pd.read_excel(ruta_entrada, sheet_name='Hoja1', header=0)
    nombres_unicos = []

    def encontrar_nombre_unico(nombre, nombres_unicos, umbral=90):
        match = process.extractOne(nombre, nombres_unicos, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= umbral:
            return match[0]
        else:
            nombres_unicos.append(nombre)
            return nombre

    df['proveedores_normalizados'] = df['Original'].apply(lambda x: encontrar_nombre_unico(x, nombres_unicos))
    df.to_excel(ruta_salida, index=False)

# Función del Proyecto Diccionario
def proyecto_diccionario(ruta_entrada, ruta_salida):
    """
    Busca palabras en un archivo Excel y las agrega en una columna separada por comas.

    Args:
    ruta_entrada (str): Ruta del archivo de entrada.
    ruta_salida (str): Ruta del archivo de salida.
    """
    archivo_excel = openpyxl.load_workbook(ruta_entrada)
    hoja1 = archivo_excel['Hoja1']
    palabras_numeros = archivo_excel['Palabras']

    lista_palabras_numeros = [celda.value for fila in palabras_numeros.iter_rows(min_col=1, max_col=1) for celda in fila]
    columna_origen = hoja1['B']
    columna_destino = hoja1['C']

    for celda_origen in columna_origen:
        contenido_celda_origen = celda_origen.value
        encontrados = [str(item) for item in lista_palabras_numeros if str(item) in str(contenido_celda_origen)]
        contenido_celda_destino = ', '.join(encontrados)
        celda_destino = columna_destino[celda_origen.row - 1]
        celda_destino.value = contenido_celda_destino

    archivo_excel.save(ruta_salida)
    archivo_excel.close()

# Función del Proyecto Referencias
def proyecto_referencias(ruta_entrada, ruta_salida):
    """
    Extrae palabras con cuatro o más números de las descripciones en el archivo Excel y guarda el resultado.

    Args:
    ruta_entrada (str): Ruta del archivo de entrada.
    ruta_salida (str): Ruta del archivo de salida.
    """
    df = pd.read_excel(ruta_entrada, sheet_name='Hoja1', header=0)

    def extraer_palabra_con_cuatro_o_mas_numeros(cadena):
        coincidencias = re.findall(r'\S*\d{4,}\S*', cadena)
        if coincidencias:
            return coincidencias[0]
        else:
            return None

    df['Cadena'] = df.apply(lambda row: ' + '.join(map(str, row[['DESC1', 'DESC2','DESC3','DESC4','DESC5']])), axis=1)
    df['Resultado'] = df['Cadena'].apply(extraer_palabra_con_cuatro_o_mas_numeros)
    df.to_excel(ruta_salida, index=False)

# Ejemplo de uso de las funciones:
# proyecto_separacion('ruta_a/SEPARACION.xlsx', 'ruta_a/SEPARACION_RESULTADO.xlsx')
# proyecto_depuracion('ruta_b/DEPURACION.xlsx', 'ruta_b/DEPURACION RESULTADO.xlsx')
# proyecto_diccionario('ruta_c/DICCIONARIO.xlsx', 'ruta_c/DICCIONARIO RESULTADO.xlsx')
# proyecto_referencias('ruta_d/APOYO II.xlsx', 'ruta_d/archivo resultado - apoyo II.xlsx')
def proyecto_contador_reemplazos():
    # Leer el archivo Excel
    df_old = pd.read_excel('C:/Users/Nicolás Ramírez/Desktop/Python/archivo_sin_espacios.xlsx', sheet_name='DatosParte1', header=0, usecols='A:E')
    
    # Filtrar las filas donde 'DESC2' no es nulo
    df = df_old[df_old['DESC2'].notnull()]
    
    # Realizar reemplazos en la columna 'DESC2'
    df['DESC2'] = df['DESC2'].str.replace('  ', ' ')
    df.replace(to_replace=r' \.', value='.', regex=True, inplace=True)
    df.replace(to_replace=r' ,', value=',', regex=True, inplace=True)
    df.replace(to_replace=r' ;', value=';', regex=True, inplace=True)
    df.replace(to_replace=r' :', value=':', regex=True, inplace=True)
    df.replace(to_replace=r' =', value='=', regex=True, inplace=True)
    df.replace(to_replace=r' /', value='/', regex=True, inplace=True)
    df.replace(to_replace=r'/ ', value='/', regex=True, inplace=True)
    df.replace(to_replace=r' -', value='-', regex=True, inplace=True)
    df.replace(to_replace=r'- ', value='-', regex=True, inplace=True)
    df.replace(to_replace=r' _', value='_', regex=True, inplace=True)
    df.replace(to_replace=r'_ ', value='_', regex=True, inplace=True)
    df.replace(to_replace=r'Á', value='A', regex=True, inplace=True)
    df.replace(to_replace=r'É', value='E', regex=True, inplace=True)
    df.replace(to_replace=r'Í', value='I', regex=True, inplace=True)
    df.replace(to_replace=r'Ó', value='O', regex=True, inplace=True)
    df.replace(to_replace=r'Ú', value='U', regex=True, inplace=True)
    df.replace(to_replace=r'P RODUCTO', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'PR ODUCTO', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'PRO DUCTO', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'PROD UCTO', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'PRODU CTO', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'PRODUC TO', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'PRODUCT O', value='PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'\. PRODUCTO', value='.PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'; PRODUCTO', value=';PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r', PRODUCTO', value=',PRODUCTO', regex=True, inplace=True)
    df.replace(to_replace=r'U NIDAD', value='UNIDAD', regex=True, inplace=True)
    df.replace(to_replace=r'UN IDAD', value='UNIDAD', regex=True, inplace=True)
    df.replace(to_replace=r'UNI DAD', value='UNIDAD', regex=True, inplace=True)
    df.replace(to_replace=r'UNID AD', value='UNIDAD', regex=True, inplace=True)
    df.replace(to_replace=r'UNIDA D', value='UNIDAD', regex=True, inplace=True)
    df.replace(to_replace=r'J UEGO', value='JUEGO', regex=True, inplace=True)
    df.replace(to_replace=r'JU EGO', value='JUEGO', regex=True, inplace=True)
    df.replace(to_replace=r'JUE GO', value='JUEGO', regex=True, inplace=True)
    df.replace(to_replace=r'JUEG O', value='JUEGO', regex=True, inplace=True)
    df.replace(to_replace=r'P IEZA', value='PIEZA', regex=True, inplace=True)
    df.replace(to_replace=r'PI EZA', value='PIEZA', regex=True, inplace=True)
    df.replace(to_replace=r'PIE ZA', value='PIEZA', regex=True, inplace=True)
    df.replace(to_replace=r'PIEZ A', value='PIEZA', regex=True, inplace=True)
    df.replace(to_replace=r'\. 00', value='.00', regex=True, inplace=True)
    df.replace(to_replace=r'\.0 0', value='.00', regex=True, inplace=True)
    df.replace(to_replace=r'\.00UNIDAD', value='.00 UNIDAD', regex=True, inplace=True)
    df.replace(to_replace=r'\.00JUEGO', value='.00 JUEGO', regex=True, inplace=True)
    df.replace(to_replace=r'\.00PIEZA', value='.00 PIEZA', regex=True, inplace=True)
    df.replace(to_replace=r'R EF', value='REF', regex=True, inplace=True)
    df.replace(to_replace=r'RE F', value='REF', regex=True, inplace=True)
    df.replace(to_replace=r'REF ERENCIA', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'REFE RENCIA', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'REFER ENCIA', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'REFERE NCIA', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'REFEREN CIA', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'REFERENC IA', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'REFERENCI A', value='REFERENCIA', regex=True, inplace=True)
    df.replace(to_replace=r'P ARTE', value='PARTE', regex=True, inplace=True)
    df.replace(to_replace=r'PA RTE', value='PARTE', regex=True, inplace=True)
    df.replace(to_replace=r'PAR TE', value='PARTE', regex=True, inplace=True)
    df.replace(to_replace=r'PART E', value='PARTE', regex=True, inplace=True)
    df.replace(to_replace=r'REFERENCIA/PARTE', value='REF/PARTE', regex=True, inplace=True)
    df.replace(to_replace=r'N RO', value='NRO', regex=True, inplace=True)
    df.replace(to_replace=r'NR O', value='NRO', regex=True, inplace=True)
    df.replace(to_replace=r'F AC', value='FAC', regex=True, inplace=True)
    df.replace(to_replace=r'FA C', value='FAC', regex=True, inplace=True)
    df.replace(to_replace=r'NROFAC', value='NRO FAC', regex=True, inplace=True)
    df.replace(to_replace=r';;NRO FAC', value=';; NRO FAC', regex=True, inplace=True)
    df.replace(to_replace=r', REF', value=',REF', regex=True, inplace=True)
    df.replace(to_replace=r'; REF', value=';REF', regex=True, inplace=True)
    df.replace(to_replace=r'\. REF', value='.REF', regex=True, inplace=True)
    df.replace(to_replace=r'C ODIGO', value='CODIGO', regex=True, inplace=True)
    df.replace(to_replace=r'CO DIGO', value='CODIGO', regex=True, inplace=True)
    df.replace(to_replace=r'COD IGO', value='CODIGO', regex=True, inplace=True)
    df.replace(to_replace=r'CODI GO', value='CODIGO', regex=True, inplace=True)
    df.replace(to_replace=r'CODIG O', value='CODIGO', regex=True, inplace=True)
    df.replace(to_replace=r'I TEM', value='ITEM', regex=True, inplace=True)
    df.replace(to_replace=r'IT EM', value='ITEM', regex=True, inplace=True)
    df.replace(to_replace=r'ITE M', value='ITEM', regex=True, inplace=True)
    df.replace(to_replace=r'\. ITEM', value='.ITEM', regex=True, inplace=True)
    df.replace(to_replace=r'; ITEM', value=';ITEM', regex=True, inplace=True)
    df.replace(to_replace=r', ITEM', value=',ITEM', regex=True, inplace=True)
    df.replace(to_replace=r'; N.C:', value=';N.C:', regex=True, inplace=True)
    df.replace(to_replace=r';N. C:', value=';N.C:', regex=True, inplace=True)
    df.replace(to_replace=r'N OMBRE', value='NOMBRE', regex=True, inplace=True)
    df.replace(to_replace(r'NO MBRE', value='NOMBRE', regex=True, inplace=True)
    df.replace(to_replace=r'NOM BRE', value='NOMBRE', regex=True, inplace=True)
    df.replace(to_replace=r'NOMB RE', value='NOMBRE', regex=True, inplace=True)
    df.replace(to_replace=r'NOMBR E', value='NOMBRE', regex=True, inplace=True)
    df.replace(to_replace=r'; NOMBRE', value=';NOMBRE', regex=True, inplace=True)
    df.replace(to_replace=r'C OMERCIAL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'CO MERCIAL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'COM ERCIAL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'COME RCIAL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'COMER CIAL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'COMERC IAL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'COMERCI AL', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'COMERCIA L', value='COMERCIAL', regex=True, inplace=True)
    df.replace(to_replace=r'R EALIZADO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'RE ALIZADO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'REA LIZADO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'REAL IZADO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'REALI ZADO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'REALIZ ADO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'REALIZA DO', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'REALIZAD O', value='REALIZADO', regex=True, inplace=True)
    df.replace(to_replace=r'T RADE', value='TRADE', regex=True, inplace=True)
    df.replace(to_replace=r'TR ADE', value='TRADE', regex=True, inplace=True)
    df.replace(to_replace=r'TRA DE', value='TRADE', regex=True, inplace=True)
    df.replace(to_replace=r'TRAD E', value='TRADE', regex=True, inplace=True)
    df.replace(to_replace=r'T RANSACTION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TR ANSACTION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRA NSACTION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRAN SACTION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANS ACTION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSA CTION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSAC TION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSAC TION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSACT ION', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSACTI ON', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSACTIO N', value='TRANSACTION', regex=True, inplace=True)
    df.replace(to_replace=r'TRANSACTION S', value='TRANSACTIONS', regex=True, inplace=True)
    df.replace(to_replace=r'R ECIBIDO', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'RE CIBIDO', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'REC IBIDO', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'RECI BIDO', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'RECIB IDO', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'RECIBI DO', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'RECIBID O', value='RECIBIDO', regex=True, inplace=True)
    df.replace(to_replace=r'DI SBONIBLE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DIS PONIBLE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DISP ONIBLE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DISPO NIBLE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DISPON IBLE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DISPONI BLE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DISPONIB LE', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'DISPONIBL E', value='DISPONIBLE', regex=True, inplace=True)
    df.replace(to_replace=r'Q U', value='QU', regex=True, inplace=True)
    df.replace(to_replace=r'QU E', value='QUE', regex=True, inplace=True)
    df.replace(to_replace=r'Q UE', value='QUE', regex=True, inplace=True)
    df.replace(to_replace=r'DE RECHO', value='DERECHO', regex=True, inplace=True)
    df.replace(to_replace=r'DER ECHO', value='DERECHO', regex=True, inplace=True)
    df.replace(to_replace=r'DERE CHO', value='DERECHO', regex=True, inplace=True)
    df.replace(to_replace=r'DEREC HO', value='DERECHO', regex=True, inplace=True)
    df.replace(to_replace=r'DERECH O', value='DERECHO', regex=True, inplace=True)
    df.replace(to_replace=r'S OLVENTE', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'SO LVENTE', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'SOL VENTE', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'SOLV ENTE', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'SOLVE NTE', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'SOLVEN TE', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'SOLVENT E', value='SOLVENTE', regex=True, inplace=True)
    df.replace(to_replace=r'E XISTENCIAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EX ISTENCIAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXI STENCIAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXIS TENCIAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXIST ENCIAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXISTE NCIAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXISTEN CIA', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXISTENC IAS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXISTENCI AS', value='EXISTENCIAS', regex=True, inplace=True)
    df.replace(to_replace=r'EXISTENCIA S', value='EXISTENCIAS', regex=True, inplace=True)

    # Obtener un listado de todos los elementos únicos en la columna 'DESC2'
    unique_elements = df['DESC2'].unique()
    
    # Contar las ocurrencias de cada elemento en la columna 'DESC2'
    element_counts = Counter(df['DESC2'])
    
    # Convertir el resultado en un DataFrame
    result_df = pd.DataFrame(element_counts.items(), columns=['Elemento', 'Ocurrencias'])
    
    # Guardar el DataFrame resultante en un nuevo archivo Excel
    result_df.to_excel('C:/Users/Nicolás Ramírez/Desktop/Python/archivo_sin_espacios_modificado.xlsx', index=False)

# Llamada a la función para ejecutar el proyecto
proyecto_contador_reemplazos()

# Función del Proyecto 1: Extraer Cantidad
def extraer_cantidad(filepath, output_filepath):
    # Cargar datos
    data = pd.read_excel(filepath, sheet_name='Sheet1', header=0)
    
    # Funciones de procesamiento
    def extraer_numeros_despues_de_cant(texto):
        patron = r"CANT ([\d]+)"
        coincidencias = re.findall(patron, texto)
        return ','.join(coincidencias)

    def extraer_numeros(texto, palabra):
        numeros = re.findall(rf'(\d+(?:[.,]\d+)?)\s{re.escape(palabra)}', texto)
        return ' ,'.join(numeros) if numeros else ''

    def llenar_resultado(row):
        if row['Analisis'] == 'Cantidad Unica' and row['Cantidad']:
            return row['Cantidad']
        elif row['Analisis'] == 'Cantidad Unica' and not row['Cantidad']:
            cantidad_2 = row['Cantidad 2']
            if '|' in cantidad_2:
                return cantidad_2.split('|')[0].split(':')[1].strip()
            elif ',' in cantidad_2:
                return cantidad_2.split(',')[0].split(':')[1].strip()
            else:
                return cantidad_2.split(':')[1].strip()
        return ''

    def analizar_fila(row):
        cantidad = row['Cantidad']
        cantidad_2 = row['Cantidad 2']
        if cantidad and ',' in cantidad:
            return "Doble Cantidad - 1"
        elif cantidad:
            return "Cantidad Unica"
        elif cantidad_2 and '|' in cantidad_2:
            return "Doble Cantidad - 2"   
        elif cantidad_2 and ',' in cantidad_2:
            return "Doble Cantidad - 2"  
        elif cantidad_2 and not any('|' in item for item in cantidad_2.split(' | ')):
            return "Cantidad Unica"      
        return "Sin Cantidad"

    df1 = {'Palabras': ['UND', 'UNID', 'PC', 'PIEZA', 'U)', 'PZA', 'SET', 'KIT', 'JUEGO', 'JGO']}
    df2 = data.copy()
    
    # Aplicar funciones
    for palabra in df1['Palabras']:
        df2[palabra] = df2['Descripción de la Mercancía Detallada 1'].apply(lambda x: extraer_numeros(x, palabra))
    
    data['Cantidad'] = data['Descripción de la Mercancía Detallada 1'].apply(extraer_numeros_despues_de_cant)
    df2['Cantidad 2'] = df2.apply(lambda row: ' | '.join(f"{col} : {row[col]}" for col in df1['Palabras'] if pd.notna(row[col]) and row[col] != ''), axis=1)
    df2.drop(columns=df1['Palabras'], inplace=True)
    df2['Analisis'] = df2.apply(analizar_fila, axis=1)
    data['Resultado'] = data.apply(llenar_resultado, axis=1)

    df_final = df2.copy()
    df_final.to_excel(output_filepath, index=False)

# Función del Proyecto 2: Husqvarna
def procesar_datos_husqvarna(filepath, output_filepath):
    # Cargar datos
    df = pd.read_excel(filepath, sheet_name='Hoja1', header=0)
    
    # Funciones de procesamiento
    def limpiar_descripcion(descripcion):
        return unidecode(descripcion).upper()

    def generar_combinaciones(palabra):
        combinaciones = []
        for espacios in itertools.product(['', ' '], repeat=len(palabra) - 1):
            combinacion = ''.join([letra + espacio for letra, espacio in zip(palabra, espacios)]) + palabra[-1]
            combinaciones.append(combinacion)
        return combinaciones

    def buscar_palabras(descripcion):
        for palabra in todas_combinaciones:
            if palabra in descripcion:
                return palabra
        return '0'
    
    # Aplicar limpieza
    df['Descripción de la Mercancía Detallada 1'] = df['Descripción de la Mercancía Detallada 1'].apply(limpiar_descripcion)
    
    # Asegurar tipos de datos
    df['Posición Arancelaria'] = pd.to_numeric(df['Posición Arancelaria'], errors='coerce')
    df['NIT del Importador'] = pd.to_numeric(df['NIT del Importador'], errors='coerce')
    df['Descripción de la Mercancía Detallada 1'] = df['Descripción de la Mercancía Detallada 1'].astype(str)
    df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors='coerce')
    
    # Crear columna CONCATENADO
    df['CONCATENADO'] = df['Posición Arancelaria'].astype(str) + '/' + df['NIT del Importador'].astype(str)
    
    # Definir grupos
    partidas_grupo_1 = [8413701100, 8424410000, 8432800000, 8433111000, 8467810000, 8467899000, 8502201000]
    concatenado_grupo1 = [
        8413701100/900322900, 8413701100/900321578, 8413701100/802019254, 8413701100/900199289, 
        8413701100/800230972, 8413701100/900561151, 8413701100/860005224, 8413701100/900438590,
        8413701100/800242106, 8413701100/890919283, 8413701100/800240559, 8413701100/860004137,
        8413701100/800021308, 8467899000/900212615, 8467899000/890921246, 8467899000/860043397,
        8467899000/900873436, 8467899000/901124835, 8467899000/900657570, 8467899000/900152232,
        8467899000/901148708, 8467899000/890926257, 8467899000/800041787, 8467899000/860062958,
        8467899000/800251163, 8467899000/900456739, 8467899000/860002175, 8467899000/860065746,
        8467899000/830116134, 8467899000/900582806, 8467899000/900319753, 8467899000/860069804,
        8467899000/860002523, 8467899000/800215562, 8467899000/890900137, 8467899000/860512728,
        8467899000/901195202, 8467899000/860001678, 8467899000/901362575, 8467899000/900172594,
        8467899000/900225518, 8467899000/890100577, 8467899000/900816058, 8467899000/860450913,
        8467899000/900842107, 8467899000/901199715, 8467899000/900431173, 8467899000/890906413,
        8467899000/890932389, 8467899000/900543486, 8467899000/800151127, 8467899000/900156499,
        8467899000/860031028, 8467899000/901194073, 8467899000/901291241, 8467810000/800015615,
        8467810000/900695457, 8467810000/901144722, 8467810000/811001550, 8467810000/830060331,
        8424410000/901156098, 8432800000/891304849, 8432800000/860016266, 8432800000/17134784,
        8432800000/901001472, 8432800000/900535938, 8433111000/700195490, 8433111000/900535938,
        8433111000/901056184, 8433111000/901352855, 8433111000/700202906, 8433111000/890801451,
        8433111000/700198691
    ]
    partidas_grupo_2 = [8467290000, 8424890090, 8479899000, 8543709000, 8467220000, 8467899000, 8424300000,
                        8467990000, 8433111000, 8433119000, 8424823000, 8430699000, 8433199000]
    
    todas_combinaciones = [generar_combinaciones(palabra) for palabra in df['Descripción de la Mercancía Detallada 1'].unique()]
    todas_combinaciones = [item for sublist in todas_combinaciones for item in sublist]

    df['Palabra Encontrada'] = df['Descripción de la Mercancía Detallada 1'].apply(buscar_palabras)

    df.to_excel(output_filepath, index=False)

# Función principal para ejecutar ambos proyectos
def ejecutar_proyectos(path_proyecto1, path_proyecto2, output_proyecto1, output_proyecto2):
    # Ejecutar Proyecto 1
    extraer_cantidad(path_proyecto1, output_proyecto1)
    
    # Ejecutar Proyecto 2
    procesar_datos_husqvarna(path_proyecto2, output_proyecto2)

# Ejemplo de cómo llamar a la función principal
path_proyecto1 = 'ruta/al/archivo_proyecto1.xlsx'
path_proyecto2 = 'ruta/al/archivo_proyecto2.xlsx'
output_proyecto1 = 'ruta/de/salida_proyecto1.xlsx'
output_proyecto2 = 'ruta/de/salida_proyecto2.xlsx'

ejecutar_proyectos(path_proyecto1, path_proyecto2, output_proyecto1, output_proyecto2)
